# 네이버 & 다음 open API 를 이용한 블로그 추출

In [ ]:
import os
import sys
import urllib
import requests
import pandas as pd
import json
import re
from bs4 import BeautifulSoup
from urllib.request import Request as req, urlopen

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = '/content/drive/My Drive/프로젝트1/df4folium.csv'

In [ ]:
jeju_site = pd.read_csv(file,encoding='utf-8')
jeju_site.head()

,가게명,주소,Latitude,Longitude
0,상춘재,제주특별자치도 제주시 조천읍 선진길 26,33.458659,126.704844
1,남양수산,제주특별자치도 서귀포시 성산읍 고성동서로56번길 11,33.450135,126.914224
2,선흘곶,제주특별자치도 제주시 조천읍 동백로 102,33.508187,126.718049
3,스시호시카이,제주특별자치도 제주시 오남로 90,33.481286,126.516769
4,미영이네식당,제주특별자치도 서귀포시 대정읍 하모항구로 42,33.217651,126.249904


In [ ]:
restaurants = '제주도 ' + jeju_site.가게명
restaurants

0           제주도 상춘재
1          제주도 남양수산
2           제주도 선흘곶
3        제주도 스시호시카이
4        제주도 미영이네식당
           ...     
251     제주도 살롱 드 라방
252    제주도 성읍 칠십리식당
253      제주도 제주시새우리
254        제주도 다정이네
255     제주도 파도소리해녀촌
Name: 가게명, Length: 256, dtype: object

In [ ]:
restaurants

0           제주도 상춘재
1          제주도 남양수산
2           제주도 선흘곶
3        제주도 스시호시카이
4        제주도 미영이네식당
           ...     
251     제주도 살롱 드 라방
252    제주도 성읍 칠십리식당
253      제주도 제주시새우리
254        제주도 다정이네
255     제주도 파도소리해녀촌
Name: 가게명, Length: 256, dtype: object

In [ ]:
for restaurant in restaurants[250:]:
  print(restaurant)

제주도 88버거
제주도 살롱 드 라방
제주도 성읍 칠십리식당
제주도 제주시새우리
제주도 다정이네
제주도 파도소리해녀촌


In [ ]:
for restaurant in restaurants[0:250]:
  print(restaurant)

제주도 상춘재
제주도 남양수산
제주도 선흘곶
제주도 스시호시카이
제주도 미영이네식당
제주도 오드랑베이커리
제주도 송림반점
제주도 피어22
제주도 카페태희
제주도 커피파인더
제주도 밀리우
제주도 엄마손횟집
제주도 숙성도
제주도 우진해장국
제주도 명진전복
제주도 신산리마을카페
제주도 곰막
제주도 국수만찬
제주도 중문수두리보말칼국수
제주도 천짓골식당
제주도 그리운바다성산포
제주도 진아떡집
제주도 애월리안
제주도 닻
제주도 미친부엌
제주도 더꽃돈
제주도 운산식당
제주도 넉둥베기
제주도 육고깃집
제주도 연리지가든
제주도 상원가든
제주도 TONKATSU 서황
제주도 하찌
제주도 표선어촌식당
제주도 범일분식
제주도 비마이게스트
제주도 제주약수터
제주도 도렐
제주도 88돼지
제주도 칠돈가
제주도 춘심이네
제주도 오는정김밥
제주도 일통이반
제주도 마마롱
제주도 미남미녀
제주도 바그다드
제주도 맛나식당
제주도 친봉산장
제주도 어우늘
제주도 섭지코지로
제주도 원더먼트제주 (휴업중)
제주도 만선식당
제주도 무거버거
제주도 포도호텔레스토랑
제주도 제주맥주
제주도 뽈살집
제주도 우유부단
제주도 더스푼
제주도 맥파이 브루어리
제주도 서양차관
제주도 은희네해장국
제주도 보롬왓 카페
제주도 No Lone Zone
제주도 제주돈아
제주도 시골길
제주도 알동네집
제주도 톰톰카레
제주도 경일낙지
제주도 민경이네어등포해녀촌
제주도 이에르바
제주도 미도리제주
제주도 올래국수
제주도 수우동
제주도 카페가호
제주도 마라도횟집
제주도 고등어쌈밥
제주도 이니스프리제주하우스
제주도 고로케야
제주도 라스 또르따스
제주도 만세국수
제주도 귤하르방
제주도 새빌카페
제주도 마고
제주도 시스터필드
제주도 해심가든
제주도 다소니
제주도 골막식당
제주도 남춘식당
제주도 도토리키친
제주도 남경미락
제주도 평대스낵
제주도 광동식당
제주도 오전열한시
제주도 섬오름해물라면
제주도 호자
제주도 플레이스엉물
제주도 광명식당
제주도 소리원
제주도 비브레이브
제주도 파라토도스
제주도 삼영식당
제주도 테라로사
제주도 유동커피
제주도 블

In [ ]:
restaurants[restaurants == '제주도 잠녀해녀촌']

249    제주도 잠녀해녀촌
Name: 가게명, dtype: object

## 1. 네이버

In [ ]:
def api_call(keyword,start):
    encText = urllib.parse.quote(keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&display=100" + "&start="+str(start)
    result = requests.get(url = url,
                         headers={"X-Naver-Client-Id":"본인 id",
                                 "X-Naver-Client-Secret":"본인 pw"})
    print(result)
    return result.json()

In [ ]:
def api_result(keyword):
    list = []
    for num in range(1,11):
      for restaurant in restaurants[0:250]:
        list = list + api_call(restaurant, num)['items'] 
    return list

In [ ]:
def api_result(keyword):
    list = []
    for num in range(1,11):
      for restaurant in restaurants[250:]:
        list = list + api_call(restaurant, num)['items'] 
    return list

In [ ]:
try:
  list = []
  result_list = api_result(restaurant)
  list = list + result_list

  file = open('jeju_1.json', 'w+')
  data = file.write(json.dumps(list))
  file.close()
except :
  list = []
  result_list = api_result(restaurant)
  list = list + result_list
  file = open('jeju_1.json', 'w+')
  data = file.write(json.dumps(list))
  file.close()

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [ ]:
len(list)

6000

In [ ]:
df = pd.read_json('jeju_1.json',encoding='euc-kr')
df['title'] = df['title'].str.rstrip().replace('(<([^>]+)>)', '',regex=True)
df['description'] = df['description'].str.rstrip().replace('(<([^>]+)>)', '',regex=True)
df.to_csv('jeju_2.csv',index=False,encoding='utf-8')

In [ ]:
pd.read_csv('jeju_1.csv').head()

,title,link,description,bloggername,bloggerlink,postdate
0,ㅣ제주도 성산여행ㅣ돌아기 제주도 여행_3 (상춘재...,https://blog.naver.com/heyan0119?Redirect=Log&...,숙소에서 한시간 안되게 달려 #상춘재 도착! 오픈 시간 전인데도 미리 손님을 받고 ...,쥬리네,https://blog.naver.com/heyan0119,20201003.0
1,"제주도 맛집 후기 (오쿠다, 서광보말칼국수, 상춘재)",https://blog.naver.com/bromiyang?Redirect=Log&...,첫날 봉순이네 흑돼지만 빼고 이번 여행에서 먹은 음식들은 다 모두 굿굿 또 가고 싶...,"함께 하는 일상, 안녕",https://blog.naver.com/bromiyang,20200731.0
2,&lt;손주들과의 제주도 여행 - 7&gt; 제주시 외곽의 식당 '상춘재',https://blog.naver.com/hansongp?Redirect=Log&l...,청와대 내에 있는 외빈을 접견하거나 식사를 대접하는 한옥 건물이 '<b>상춘재</b...,한송의 세상보기,https://blog.naver.com/hansongp,20200817.0
3,제주도 상춘재에서 먹어보는 전복 돌솥비빔밥 맛집,https://blog.naver.com/gusrudrla98?Redirect=Lo...,<b>제주도 상춘재</b> 전복 돌솥비빔밥 맛집 제주도 하면 고기국수? 각종 해산물...,일탈이야기,https://blog.naver.com/gusrudrla98,20201004.0
4,제주도 동쪽 맛집 :: 제주 조천읍 맛집 - &quot;상춘재&quot; 청와대 셰...,https://blog.naver.com/lovely__ban?Redirect=Lo...,제주도 동쪽 맛집 :: 제주 조천읍 맛집 - &quot;상춘재&quot; 청와대 셰...,LOVELY.BAN ✨,https://blog.naver.com/lovely__ban,20200709.0


In [ ]:
df.to_csv('/content/drive/My Drive/프로젝트1/jeju_blog_1.csv',index=False,encoding='utf-8')

In [ ]:
df.to_csv('/content/drive/My Drive/프로젝트1/jeju_blog_2.csv',index=False,encoding='utf-8')

In [ ]:
# 나머지 6개 데이터 가져온것 csv로 만들어주기

In [ ]:
file1 = '/content/drive/My Drive/프로젝트1/jeju_blog_1.csv'
file2 = '/content/drive/My Drive/프로젝트1/jeju_blog_2.csv'

In [ ]:
df1 = pd.read_csv(file1,encoding='utf-8')
df1['description'] = df1['description'].str.rstrip().replace('(<([^>]+)>)', '',regex=True)
df1.head()

,title,link,description,bloggername,bloggerlink,postdate
0,ㅣ제주도 성산여행ㅣ돌아기 제주도 여행_3 (상춘재...,https://blog.naver.com/heyan0119?Redirect=Log&...,숙소에서 한시간 안되게 달려 #상춘재 도착! 오픈 시간 전인데도 미리 손님을 받고 ...,쥬리네,https://blog.naver.com/heyan0119,20201003.0
1,"제주도 맛집 후기 (오쿠다, 서광보말칼국수, 상춘재)",https://blog.naver.com/bromiyang?Redirect=Log&...,첫날 봉순이네 흑돼지만 빼고 이번 여행에서 먹은 음식들은 다 모두 굿굿 또 가고 싶...,"함께 하는 일상, 안녕",https://blog.naver.com/bromiyang,20200731.0
2,&lt;손주들과의 제주도 여행 - 7&gt; 제주시 외곽의 식당 '상춘재',https://blog.naver.com/hansongp?Redirect=Log&l...,"청와대 내에 있는 외빈을 접견하거나 식사를 대접하는 한옥 건물이 '상춘재' 인데, ...",한송의 세상보기,https://blog.naver.com/hansongp,20200817.0
3,제주도 상춘재에서 먹어보는 전복 돌솥비빔밥 맛집,https://blog.naver.com/gusrudrla98?Redirect=Lo...,제주도 상춘재 전복 돌솥비빔밥 맛집 제주도 하면 고기국수? 각종 해산물? 보통 제주...,일탈이야기,https://blog.naver.com/gusrudrla98,20201004.0
4,제주도 동쪽 맛집 :: 제주 조천읍 맛집 - &quot;상춘재&quot; 청와대 셰...,https://blog.naver.com/lovely__ban?Redirect=Lo...,제주도 동쪽 맛집 :: 제주 조천읍 맛집 - &quot;상춘재&quot; 청와대 셰...,LOVELY.BAN ✨,https://blog.naver.com/lovely__ban,20200709.0


In [ ]:
df2 = pd.read_csv(file2,encoding='utf-8')
df2['description'] = df2['description'].str.rstrip().replace('(<([^>]+)>)', '',regex=True)
df2.head()

,title,link,description,bloggername,bloggerlink,postdate
0,제주도 꼭 가봐야할 88버거와 제주 아쿠아플라넷,https://blog.naver.com/ksh3820?Redirect=Log&lo...,"구경을 끝마치고 끼니를 해결하기 위해 88버거를 방문했는데요, 매장은 아쿠아리움 안...",달리의 USB,https://blog.naver.com/ksh3820,20191118
1,"제주도 뚜벅이 여행 2일차, 서귀포 88버거 오는정김밥 예약",https://blog.naver.com/hnimmm?Redirect=Log&log...,Previous image Next image 제주도 서귀포 88버거 메뉴판 커플 ...,내 오늘의 모든 리뷰,https://blog.naver.com/hnimmm,20200902
2,서귀포 맛집 : 제주도 수제버거 88버거 (흑돼지햄버거),https://blog.naver.com/hotjewel?Redirect=Log&l...,그렇게 테이크아웃을 해서 숙소로 가져온 88버거는.. 샤워를 하고나온 오빠의 맥주안...,Just 켄다,https://blog.naver.com/hotjewel,20191029
3,&quot;88버거&quot; 서귀포 올레시장... 버거 맛집 솔직후기 [제주도...,https://blog.naver.com/clairlee77?Redirect=Log...,두툼한 컷의 후렌치후라이가 더 맛있었던 것 같아요 가성비와 비주얼에 비해 햄버거의 ...,A Bite of Happiness,https://blog.naver.com/clairlee77,20200930
4,"190102~190105 제주도여행 (3)... 1100고지, 88버거...",https://blog.naver.com/meeso101?Redirect=Log&l...,전 날과는 아예 다른 하늘을 보여준 제주도 그래도 감사하고! 파티 때 언니가 제주도...,Summer,https://blog.naver.com/meeso101,20190301


In [ ]:
# 두개의 csv합치기

In [ ]:
blog_naver = pd.concat([df1,df2])

In [ ]:
blog_naver.to_csv('/content/drive/My Drive/프로젝트1/blog_naver.csv',index=False,encoding='utf-8')

In [ ]:
# link주소로 블로그 전문 크롤링하기

In [ ]:
files = '/content/drive/My Drive/프로젝트1/blog_naver.csv'
blog_naver = pd.read_csv(files,encoding='utf-8')
blog_naver.head()

,title,link,description,bloggername,bloggerlink,postdate
0,ㅣ제주도 성산여행ㅣ돌아기 제주도 여행_3 (상춘재...,https://blog.naver.com/heyan0119?Redirect=Log&...,숙소에서 한시간 안되게 달려 #상춘재 도착! 오픈 시간 전인데도 미리 손님을 받고 ...,쥬리네,https://blog.naver.com/heyan0119,20201003.0
1,"제주도 맛집 후기 (오쿠다, 서광보말칼국수, 상춘재)",https://blog.naver.com/bromiyang?Redirect=Log&...,첫날 봉순이네 흑돼지만 빼고 이번 여행에서 먹은 음식들은 다 모두 굿굿 또 가고 싶...,"함께 하는 일상, 안녕",https://blog.naver.com/bromiyang,20200731.0
2,&lt;손주들과의 제주도 여행 - 7&gt; 제주시 외곽의 식당 '상춘재',https://blog.naver.com/hansongp?Redirect=Log&l...,"청와대 내에 있는 외빈을 접견하거나 식사를 대접하는 한옥 건물이 '상춘재' 인데, ...",한송의 세상보기,https://blog.naver.com/hansongp,20200817.0
3,제주도 상춘재에서 먹어보는 전복 돌솥비빔밥 맛집,https://blog.naver.com/gusrudrla98?Redirect=Lo...,제주도 상춘재 전복 돌솥비빔밥 맛집 제주도 하면 고기국수? 각종 해산물? 보통 제주...,일탈이야기,https://blog.naver.com/gusrudrla98,20201004.0
4,제주도 동쪽 맛집 :: 제주 조천읍 맛집 - &quot;상춘재&quot; 청와대 셰...,https://blog.naver.com/lovely__ban?Redirect=Lo...,제주도 동쪽 맛집 :: 제주 조천읍 맛집 - &quot;상춘재&quot; 청와대 셰...,LOVELY.BAN ✨,https://blog.naver.com/lovely__ban,20200709.0


## 2.다음

In [ ]:
def kakao_api_call(keyword,page):
    encText = urllib.parse.quote(keyword)
    url = "https://dapi.kakao.com/v2/search/blog?query="+ encText +"&size=50" + "&page="+str(page)
    result = requests.get(url = url,
                         headers={'Authorization': 'KakaoAK 본인키'})
    print(result)
    return result.json()

In [ ]:
def kakao_api_result(keyword):
    blog_list = []
    for num in range(1,7):
      for restaurant in restaurants:
        blog_list = blog_list + kakao_api_call(restaurant, num)['documents'] 
    return blog_list

In [ ]:
blog_list = []
blog_result = kakao_api_result(restaurant)
blog_list = blog_list + blog_result

file = open('jeju_3.json', 'w+')
data = file.write(json.dumps(blog_list))
file.close()

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [ ]:
len(blog_list)

76263

In [ ]:
df3 = pd.read_json('jeju_3.json',encoding='euc-kr')
df3['title'] = df3['title'].str.rstrip().replace('(<([^>]+)>)', '',regex=True)
df3['contents'] = df3['contents'].str.rstrip().replace('(<([^>]+)>)', '',regex=True)
df3.to_csv('jeju_3.csv',index=False,encoding='utf-8')

In [ ]:
pd.read_csv('jeju_3.csv').head()

,blogname,contents,datetime,thumbnail,title,url
0,대들보부상,"과감히 넣고 풍림다방 혹은 세화해변근처 한라산카페, 카페오길 등 추천합니다? 그럼 ...",2020-05-01 02:06:00+09:00,https://search3.kakaocdn.net/argon/130x130_85_...,🍽 제주도 최고맛집 상춘재,http://supermj.tistory.com/19
1,벨라봉봉,제주도 상춘재 비빔밥 너무 맛나요ㅠㅠ ​ 저번주에 남편이랑 즐겁게 제주도 여행을 다...,2020-06-28 21:32:00+09:00,https://search1.kakaocdn.net/argon/130x130_85_...,제주도 상춘재 비빔밥 너무 맛나요ㅠㅠ,http://blog.naver.com/bellabonbon/222014990393
2,일탈이야기,각종 해산물? ​ 보통 제주도하면 저는 고기국수나 아니면 해산물이 생각나는 것 같네...,2020-10-04 08:35:00+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,제주도 상춘재에서 먹어보는 전복 돌솥비빔밥 맛집,http://blog.naver.com/gusrudrla98/222106188767
3,여행이좋다,제주도 숨은 맛집 제주 상춘재 청와대 출신의 주방장이 음식을 만드는 곳이 있어다고 ...,2018-01-09 07:00:00+09:00,https://search3.kakaocdn.net/argon/130x130_85_...,제주도 숨은 맛집 제주 상춘재 비빔밥,http://noproblemyourlife.tistory.com/2372
4,마카롱의 일상다반사,휴무구요. 주말에는 정말 사람들이 많다고 합니다. 특히 어르신들을 모시고 제주도에 ...,2019-05-25 16:04:00+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,제주도 조천읍 맛집 &#34;상춘재&#34;,http://blog.naver.com/makafood/221546344034


In [ ]:
df3.to_csv('/content/drive/My Drive/프로젝트1/blog_kakao.csv',index=False,encoding='utf-8')

In [ ]:
file_kakao = '/content/drive/My Drive/프로젝트1/blog_kakao.csv'
blog_kakao = pd.read_csv(file_kakao,encoding='utf-8')
blog_kakao.head()

,blogname,contents,datetime,thumbnail,title,url
0,대들보부상,"과감히 넣고 풍림다방 혹은 세화해변근처 한라산카페, 카페오길 등 추천합니다? 그럼 ...",2020-05-01 02:06:00+09:00,https://search3.kakaocdn.net/argon/130x130_85_...,🍽 제주도 최고맛집 상춘재,http://supermj.tistory.com/19
1,벨라봉봉,제주도 상춘재 비빔밥 너무 맛나요ㅠㅠ ​ 저번주에 남편이랑 즐겁게 제주도 여행을 다...,2020-06-28 21:32:00+09:00,https://search1.kakaocdn.net/argon/130x130_85_...,제주도 상춘재 비빔밥 너무 맛나요ㅠㅠ,http://blog.naver.com/bellabonbon/222014990393
2,일탈이야기,각종 해산물? ​ 보통 제주도하면 저는 고기국수나 아니면 해산물이 생각나는 것 같네...,2020-10-04 08:35:00+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,제주도 상춘재에서 먹어보는 전복 돌솥비빔밥 맛집,http://blog.naver.com/gusrudrla98/222106188767
3,여행이좋다,제주도 숨은 맛집 제주 상춘재 청와대 출신의 주방장이 음식을 만드는 곳이 있어다고 ...,2018-01-09 07:00:00+09:00,https://search3.kakaocdn.net/argon/130x130_85_...,제주도 숨은 맛집 제주 상춘재 비빔밥,http://noproblemyourlife.tistory.com/2372
4,마카롱의 일상다반사,휴무구요. 주말에는 정말 사람들이 많다고 합니다. 특히 어르신들을 모시고 제주도에 ...,2019-05-25 16:04:00+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,제주도 조천읍 맛집 &#34;상춘재&#34;,http://blog.naver.com/makafood/221546344034
